In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from scipy.special import softmax
# !pip install googletrans==4.0.0-rc1
from googletrans import Translator

In [3]:
vk_data = pd.read_excel("VK_full.xlsx")
rbc_data = pd.read_excel("rbc_full_crypto_articles_to_2024-05-24.xlsx")

# Предобработаем данные

### Для начала уберем NaN

In [4]:
vk_data.isnull().sum()


Уникальный идентификатор записи       0
Дата и время публикации               0
Текст записи                       1886
Количество комментариев               0
Информация о лайках                   0
Количество просмотров                 0
dtype: int64

In [5]:

vk_data = vk_data.dropna(subset=['Текст записи'])


In [6]:
vk_data.isnull().sum()

Уникальный идентификатор записи    0
Дата и время публикации            0
Текст записи                       0
Количество комментариев            0
Информация о лайках                0
Количество просмотров              0
dtype: int64

In [7]:
rbc_data.isnull().sum()

Unnamed: 0        0
0                 2
1                 0
2                 0
3                 0
4                 0
5             12375
6                 0
7                 0
8                 0
9                 0
10                0
11             2359
12            12375
13                0
14            12375
15                0
16                0
17                0
dtype: int64

In [8]:
rbc_data = rbc_data.iloc [: , 2:]
rbc_data = rbc_data[1:]
rbc_data.reset_index(drop= True , inplace= True)
rbc_data = rbc_data.drop(index = 4795)
rbc_data.reset_index(drop= True , inplace= True)

In [9]:
rbc_data = rbc_data.rename(columns={1: 'id', 2: 'project', 3: 'project_nick', 4: 'type', 5: 'category', 6: 'title',7 : 'body', 8: 'publish_date', 9: 'publish_date_t', 10: 'fronturl', 11: 'picture', 12: 'badge', 13: 'pay_option', 14: 'data', 15: '_score', 16: 'overview', 17: 'text'})

In [10]:
rbc_data['publish_date'] = pd.to_datetime(rbc_data['publish_date']).dt.date

In [11]:
vk_data['Дата и время публикации'] = pd.to_datetime(vk_data['Дата и время публикации']).dt.date

In [12]:
rbc_data.isnull().sum()

id                    0
project               0
project_nick          0
type                  0
category          12375
title                 0
body                  0
publish_date          0
publish_date_t        0
fronturl              0
picture            2359
badge             12375
pay_option            0
data              12375
_score                0
overview              0
text                  0
dtype: int64

# проверка видеокарты

In [13]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3070


##### мне вывело
##### "There are 1 GPU(s) available.
##### We will use the GPU: NVIDIA GeForce RTX 3070"
##### Если у вас не подключена видеокарта, то скачайте torch по полной ссылке на сайте https://pytorch.org/

# Модель RUBERT и классификация

Модель 0 # скипнул

In [14]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("r1char9/rubert-base-cased-russian-sentiment")
# model = AutoModelForSequenceClassification.from_pretrained("r1char9/rubert-base-cased-russian-sentiment")

Модель 1

0: POSITIVE
1: NEGATIVE

In [18]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-mokoron')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-mokoron', return_dict=True)


In [19]:
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    #predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

начнем с рбк

In [100]:
rbk_postive = []
rbk_negative = []
for i in range(len(rbc_data['title'])):
    tensorr = predict(rbc_data['title'][i])
    rbk_postive.append(tensorr[0][0].item()) # в список добавляется предсказанное позитивное значение
    rbk_negative.append(tensorr[0][1].item()) # в список добавляется предсказанное негативное значение
#Позитивное = 1 - негативное

In [ ]:
rbc_data['rbk_positive'] = rbk_postive
rbc_data['rbk_negative'] = rbk_negative

In [ ]:
columns_to_save = ['publish_date', 'rbk_positive', 'rbk_negative']

In [ ]:
rbc_data.to_csv('rbk_RuBeart_Titles_Sentiment.csv', columns=columns_to_save, index=False)

теперь вк

In [28]:
vk_data.reset_index(drop= True , inplace= True)

In [32]:
vk_postive = []
vk_negative = []
for i in range(len(vk_data['Текст записи'])):
    tensorr = predict(vk_data['Текст записи'][i])
    vk_postive.append(tensorr[0][0].item())
    vk_negative.append(tensorr[0][1].item())

In [37]:
vk_data['vk_posotive_rubert'] = vk_postive
vk_data['vk_negative_rubert'] = vk_negative

In [39]:
columns_to_save_VK = ['Дата и время публикации', 'vk_posotive_rubert', 'vk_negative_rubert']
vk_data.to_csv('vk_RuBeart_text_Sentiment.csv', columns=columns_to_save_VK, index=False)

In [ ]:
vk_data = vk_data.rename(columns={'Текст записи': 'Post text'})

# Модель CRYPTO_BERT 

для него нужен английский текст

Скачиваю криптоберта

In [14]:
# Load model directly 
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ElKulako/cryptobert")
model = AutoModelForSequenceClassification.from_pretrained("ElKulako/cryptobert")

Перевод текста

текст я перевожу в отдельном файле translator.py здесь я просто сохраняю и получаю csv файл


In [42]:
columns_to_save_for_VK_translator = ['Post text']

In [48]:
vk_data.to_csv('vk_text.csv', columns=columns_to_save_for_VK_translator, index=True)

In [182]:
columns_to_save_for_RBK_translator = ['title']

In [183]:
rbc_data.to_csv('rbk_text.csv', columns=columns_to_save_for_RBK_translator, index=True)

### После перевода

In [237]:
# Импортируем CSV файл в DataFrame
trans_rbc_titles = pd.read_csv("translated_rbc.csv")

In [238]:
trans_vk_text = pd.read_csv("translated_vk_text.csv")

Итак вот наш переведенный текст

In [239]:
trans_vk_text['Post date'] = vk_data['Дата и время публикации']

In [240]:
trans_vk_text

,id,Post text,Post date
0,0,We follow everything that happens in the world...,2022-10-19
1,1,The Central Bank of the Russian Federation sta...,2024-05-24
2,2,"According to a Bloomberg analyst, the SEC will...",2024-05-23
3,3,"The AI ​​sector is on everyone's lips, what to...",2024-05-23
4,4,"In Russia, they considered a dispute about the...",2024-05-23
...,...,...,...
12507,12507,money-clash.ru ⛔SCAM! DO NOT INVEST.\n⛔⛔⛔The a...,2018-01-06
12508,12508,Every well-built house began with a plan. \nNa...,2018-01-06
12509,12509,"For smart people, money is a means; for fools,...",2018-01-06
12510,12510,Our big flaw is that we give up too quickly. T...,2018-01-06


In [241]:
trans_rbc_titles


,ind,title
0,0,The most famous cloud mining service confirmed...
1,1,No time to explain: everything about cryptocur...
2,2,You might have missed it: exchanges will add S...
3,3,A major exchange has returned to work after a ...
4,4,There will be no public ICO round. All details...
...,...,...
12370,12370,The US approved the launch of exchange-traded ...
12371,12371,The main trends of the cryptocurrency market i...
12372,12372,Experts called the Solana cryptocurrency suita...
12373,12373,What will happen to the Ethereum rate after ET...


Обнаружил проблемы с вк датасетом

In [242]:
# обнаружил проблему, что некоторые строчки не перевелись, решаю ее удаляя эти строчки кодом ниже

In [243]:
import re
def contains_russian(text):
    if isinstance(text, str):
        return bool(re.search('[а-яА-Я]', text))
    return False
# Применим функцию ко всем столбцам DataFrame
mask = trans_vk_text.applymap(contains_russian).any(axis=1)

# Удаляем строки, где есть русские буквы
vk_trans_cleaned = trans_vk_text[~mask]
vk_trans_cleaned.reset_index(drop= True , inplace= True)

C:\Users\User\AppData\Local\Temp\ipykernel_31060\1458871707.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = trans_vk_text.applymap(contains_russian).any(axis=1)


In [244]:
# Также обнаружил проблему, что некоторые строчки имеют тип float, решаю ее удаляя эти строчки кодом ниже

In [245]:
def contains_float(value):
    return isinstance(value, float)

mask = vk_trans_cleaned.applymap(contains_float).any(axis=1)

# Удаляем строки, где есть значения типа float
vk_trans_cleaned = vk_trans_cleaned[~mask]
vk_trans_cleaned.reset_index(drop= True , inplace= True)

C:\Users\User\AppData\Local\Temp\ipykernel_31060\967498190.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = vk_trans_cleaned.applymap(contains_float).any(axis=1)


In [246]:
# Также приходится удалять совсем длинные посты (>1700 символов), так как с ними не справляется cryptobert 

In [247]:
ct = 0
for i in range(len(vk_trans_cleaned['Post text'])):
    if len(str(vk_trans_cleaned['Post text'][i])) > ct:
        ct = len(str(vk_trans_cleaned['Post text'][i]))
        
print(ct)        

for i in range(len(vk_trans_cleaned['Post text'])):
    if len(str(vk_trans_cleaned['Post text'][i])) > 1400:
        vk_trans_cleaned = vk_trans_cleaned.drop(i) 
   
len(vk_trans_cleaned['Post text'])
vk_trans_cleaned.reset_index(drop= True , inplace= True)

ct = 0
for i in range(len(vk_trans_cleaned['Post text'])):
    if len(str(vk_trans_cleaned['Post text'][i])) > ct:
        ct = len(str(vk_trans_cleaned['Post text'][i]))
        
print(ct)   # ct показатель максимальной длины поста в списке

4901
1400


In [248]:
print("Итого было VK постов: " + str(len(trans_vk_text)))
print("Стало VK постов: " + str(len(vk_trans_cleaned)))

Итого было VK постов: 12512
Стало VK постов: 9899


Начнем с РБК


In [249]:
def predict(text):
    inputs = tokenizer(text, max_length=1900, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    #predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [253]:
rbk_postive_cryptobert = []
rbk_negative_cryptobert = []
rbk_neutral_cryptobert = []

for i in range(len(trans_rbc_titles)):
    tensorr = predict(trans_rbc_titles['title'][i])
    rbk_postive_cryptobert.append(tensorr[0][2].item())
    rbk_negative_cryptobert.append(tensorr[0][0].item())
    rbk_neutral_cryptobert.append(tensorr[0][1].item())

In [33]:
len(rbk_postive_cryptobert)

12375

In [34]:
rbc_data['rbk_postive_cryptobert'] = rbk_postive_cryptobert
rbc_data['rbk_negative_cryptobert'] = rbk_negative_cryptobert
rbc_data['rbk_neutral_cryptobert'] = rbk_neutral_cryptobert 

In [ ]:
columns_to_save = ['publish_date', 'rbk_postive_cryptobert', 'rbk_negative_cryptobert', 'rbk_neutral_cryptobert']

In [ ]:
rbc_data.to_csv('rbk_data_from_cryptobert.csv', columns=columns_to_save, index=False)

Продолжим с ВК 

In [251]:
def predict(text):
    inputs = tokenizer(text, max_length=1500, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    #predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

In [252]:
vk_postive_cryptobert = []
vk_negative_cryptobert = []
vk_neutral_cryptobert = []
for i in range(len(vk_trans_cleaned)):
    tensorr = predict((vk_trans_cleaned['Post text'][i]))
    vk_postive_cryptobert.append(tensorr[0][2].item())
    vk_negative_cryptobert.append(tensorr[0][0].item())
    vk_neutral_cryptobert.append(tensorr[0][1].item())

In [254]:
len(vk_postive_cryptobert)

9899

In [258]:
vk_trans_cleaned['vk_postive_cryptobert'] = vk_postive_cryptobert
vk_trans_cleaned['vk_negative_cryptobert'] = vk_negative_cryptobert
vk_trans_cleaned['vk_neutral_cryptobert'] = vk_neutral_cryptobert 


In [259]:
vk_trans_cleaned = vk_trans_cleaned.rename(columns={'Post date': 'publish_date'})

In [260]:
vk_trans_cleaned

,id,Post text,publish_date,vk_postive_cryptobert,vk_negative_cryptobert,vk_neutral_cryptobert
0,0,We follow everything that happens in the world...,2022-10-19,0.106720,0.000120,0.893160
1,1,The Central Bank of the Russian Federation sta...,2024-05-24,0.334370,0.023998,0.641632
2,3,"The AI ​​sector is on everyone's lips, what to...",2024-05-23,0.662217,0.001338,0.336445
3,4,"In Russia, they considered a dispute about the...",2024-05-23,0.265329,0.010629,0.724042
4,5,The creator of the meme token set himself on f...,2024-05-23,0.202640,0.583108,0.214252
...,...,...,...,...,...,...
9894,12507,money-clash.ru ⛔SCAM! DO NOT INVEST.\n⛔⛔⛔The a...,2018-01-06,0.132776,0.464834,0.402390
9895,12508,Every well-built house began with a plan. \nNa...,2018-01-06,0.488020,0.001154,0.510826
9896,12509,"For smart people, money is a means; for fools,...",2018-01-06,0.636082,0.000986,0.362932
9897,12510,Our big flaw is that we give up too quickly. T...,2018-01-06,0.675413,0.000109,0.324478


In [ ]:
columns_to_save = ['publish_date', 'vk_postive_cryptobert', 'vk_negative_cryptobert', 'vk_neutral_cryptobert']
vk_trans_cleaned.to_csv('vk_CryptoBert_text_Sentiment.csv', columns=columns_to_save, index=False)

"Bearish" : 0, "Neutral": 1, "Bullish": 2

## Готово (получен датасет с криптоберта)